# Datascientist_selary_analysis
The data is downloaded from kaggle(https://www.kaggle.com/kaggle/kaggle-survey-2017), our purpose is to analysis the salary of Datascientist around world.

## Dispose of the data
The original data is contain 16716 respondents from about 52 countrys. However, some important information is absent for some of data, we need to select a part of it to achieve our purpose.

**Import some necessary tools for data pre-processing and load the data to pandas including exchange rate of different currency to USD.**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

data = pd.read_csv("../input/multipleChoiceResponses.csv", encoding = "ISO-8859-1")
chang_rate = pd.read_csv("../input/conversionRates.csv", encoding = "ISO-8859-1")

**Combine with this to dataframe and select the features we needed.**

In [ ]:
data_merged = pd.merge(data,chang_rate, how = 'left',left_on='CompensationCurrency', right_on='originCountry' )
data_usefull = data_merged[['GenderSelect','Country', 'Age','EmploymentStatus','CodeWriter',
             'CurrentJobTitleSelect', 'CurrentEmployerType', 'LanguageRecommendationSelect',
             'FormalEducation', 'CompensationAmount', 'CompensationCurrency', 'exchangeRate']]

**Select the data with full-time job and delete the data without the information of Countray or salay.**

In [ ]:
data_selected = data_usefull[data_usefull['EmploymentStatus']=='Employed full-time']
data_selected = data_selected.dropna(axis=0, how = 'any', subset=['Country', 'CompensationAmount','exchangeRate'])

**Change the string to float and exchange all salary into USD.**

In [ ]:
data_selected['CompensationAmount'] = data_selected['CompensationAmount'].apply(lambda x: x.replace(',',''))
data_selected.drop(data_selected[data_selected['CompensationAmount']=='-'].index,inplace=True)
data_selected['CompensationAmountExanged'] = data_selected.apply(lambda x : float(x['CompensationAmount'])*float(x['exchangeRate']),  axis=1)

**Replace the other name of China into "Chian" especially including “Taiwan”.**

In [ ]:
data_selected.replace({"People 's Republic of China":'China' , "Republic of China":'China', "Taiwan":'China'}, inplace = True)

**Show some basic Information**

In [ ]:
data_selected.head()

## plot

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#Import Plotly and use it in the Offline Mode
import plotly
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.tools as tls
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as fig_fact
plotly.tools.set_config_file(world_readable=True, sharing='public')

**Gender**

In [ ]:
sns.countplot(y='GenderSelect', data=data_selected)
plt.show()

**Country**

In [ ]:
data_country = data_selected.drop(data_selected[data_selected['Country']=='Other'].index)['Country'].value_counts().to_frame()
sns.barplot(data_country['Country'][:15],data_country.index[:15],palette='inferno')
plt.title('Top 15 Countries by number of respondents')
plt.xlabel('')
fig=plt.gcf()
fig.set_size_inches(10,10)
plt.show()
data_country[:15]

In [ ]:
data = [ dict(
        type = 'choropleth',
        locations = data_country.index,
        locationmode = 'country names',
        z = data_country['Country'],
        text = data_country.index,
        colorscale = [[0,'rgb(255, 255, 255)'],[1,'rgb(56, 142, 60)']],
        autocolorscale = False,
        reversescale = False,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            tickprefix = '',
            title = 'Survey Respondents'),
      ) ]

layout = dict(
    title = 'Survey Respondents by Nationality',
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='survey-world-map')

**Age**

In [ ]:
#Plot the Age distribution
fig = fig_fact.create_distplot([data_selected[data_selected['Age'] > 0]['Age']], ['age'], colors=['#BA68C8'])
py.iplot(fig, filename='Basic Distplot')

**Salary**

In [ ]:
plt.subplots(figsize=(15,8))
data_salary=data_selected[data_selected['CompensationAmountExanged']<1000000]
sns.distplot(data_salary['CompensationAmountExanged'])
plt.title('Salary Distribution',size=15)
plt.show()

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
sal_coun=data_salary.groupby('Country')['CompensationAmountExanged'].median().sort_values(ascending=False)[:15].to_frame()
sns.barplot('CompensationAmountExanged',sal_coun.index,data=sal_coun,palette='RdYlGn',ax=ax[0])
ax[0].axvline(data_salary['CompensationAmountExanged'].median(),linestyle='dashed')
ax[0].set_title('Highest Salary Paying Countries')
ax[0].set_xlabel('')
resp_coun=data_country[:15]
max_coun=data_salary.groupby('Country')['CompensationAmountExanged'].median().to_frame()
max_coun=max_coun[max_coun.index.isin(resp_coun.index)]
max_coun.sort_values(by='CompensationAmountExanged',ascending=True).plot.barh(width=0.8,ax=ax[1],color=sns.color_palette('RdYlGn'))
ax[1].axvline(data_salary['CompensationAmountExanged'].median(),linestyle='dashed')
ax[1].set_title('Compensation of Top 15 Respondent Countries')
ax[1].set_xlabel('')
ax[1].set_ylabel('')
plt.subplots_adjust(wspace=0.8)
plt.show()

In [ ]:
plt.subplots(figsize=(10,8))
sns.boxplot(y='GenderSelect',x='CompensationAmountExanged',data=data_salary)
plt.ylabel('')
plt.show()

In [ ]:
sal_job=data_salary.groupby('CurrentJobTitleSelect')['CompensationAmountExanged'].median().to_frame().sort_values(by='CompensationAmountExanged',ascending=False)
ax=sns.barplot(sal_job.CompensationAmountExanged,sal_job.index,palette=sns.color_palette('inferno',20))
plt.title('Compensation By Job Title',size=15)
for i, v in enumerate(sal_job.CompensationAmountExanged): 
    ax.text(.5, i, v,fontsize=10,color='white',weight='bold')
fig=plt.gcf()
fig.set_size_inches(8,8)
plt.show()


In [ ]:
data_plot = data_selected[data_selected['Country'].isin(data_selected['Country'].value_counts().index[0:5].tolist())]
data_plot = data_plot[data_plot['CurrentJobTitleSelect'].isin(data_plot['CurrentJobTitleSelect'].value_counts().index[0:4].tolist())]
sns.set(style="whitegrid")

# Draw a nested barplot to show survival for class and sex
g = sns.factorplot(x="Country", y="CompensationAmountExanged", hue="CurrentJobTitleSelect", data=data_plot,
                size=12, kind="bar", palette="muted")
g.despine(left=True)
g.set_ylabels("survival probability")
plt.show()

In [ ]:
data_plot = data_selected[data_selected['Country'].isin(data_selected['Country'].value_counts().index[0:4].tolist())]
data_plot = data_plot[data_plot['CurrentJobTitleSelect'].isin(data_plot['CurrentJobTitleSelect'].value_counts().index[0:4].tolist())]
g = sns.FacetGrid(data_plot, row="CurrentJobTitleSelect", col="Country", margin_titles=True)
bins = np.linspace(0, 120000, 1000)
g.map(plt.hist, "CompensationAmountExanged", color="steelblue", bins=bins)
plt.show()

In [ ]:
data_plot = data_selected[data_selected['Country'].isin(data_selected['Country'].value_counts().index[0:15].tolist())]
data_plot = data_plot[data_plot['CurrentJobTitleSelect'].isin(data_plot['CurrentJobTitleSelect'].value_counts().index[0:10].tolist())]
fig = plt.figure(figsize=(20, 8))
sns.set(style="darkgrid")
ax = sns.countplot(y="Country",  hue="CurrentJobTitleSelect",  data=data_plot)
plt.show()

In [ ]:
data_plot = data_selected[data_selected['Country'].isin(data_selected['Country'].value_counts().index[0:10].tolist())]
data_plot = data_plot[data_plot['CurrentJobTitleSelect'].isin(data_plot['CurrentJobTitleSelect'].value_counts().index[0:10].tolist())]
plt.subplots(figsize=(25,35))
useful = data_plot['Country'].value_counts().index.tolist()
length=len(data_plot['Country'].value_counts())
for i,j in zip(useful,range(length)):
    plt.subplot((length/2),3,j+1)
    plt.subplots_adjust(wspace=0.2,hspace=0.2)
    data_plot[data_plot['Country'] == i]['CurrentJobTitleSelect'].value_counts().plot.pie(autopct='%1.1f%%',wedgeprops = { 'linewidth' : 2, 'edgecolor' : 'white' })
    plt.title(i,size=25)
    my_circle=plt.Circle( (0,0), 0.7, color='white')
    p=plt.gcf()
    p.gca().add_artist(my_circle)
    plt.xlabel('')
    plt.ylabel('')
plt.show()

In [ ]:
*Refrence*
1. https://www.kaggle.com/rounakbanik/data-science-faq
2. https://www.kaggle.com/ash316/novice-to-grandmaster
3. https://www.kaggle.com/hakkisimsek/plotly-tutorial-1